# Psychometric function

In [ ]:
def roc(img_base, img_new, threshold):

    fpr = []
    tpr = []

    yes_s = sum([1 if i >= threshold else 0 for i in img_new]) #tp 
    no_s = sum([1 if i < threshold else 0 for i in img_new]) #fn

    #No stimul
    yes_s0 = sum([1 if i >= threshold else 0 for i in img_base]) #fp
    no_s0 = sum([1 if i < threshold else 0 for i in img_base]) #tn
    
    print('Number of correct answers: ', yes_s + no_s0)

    print('Number of wrong answers: ', yes_s0 + no_s)

    tp = yes_s
    fn = no_s
    tn = no_s0
    fp = yes_s0

    fpr.append(fp / (fp + tn))
    tpr.append(tp / (tp + fn))  

    return [fpr, tpr]

In [ ]:
mu_s = [np.mean(Data_df[Data_df.Image == images[i+1]].norm_Rating) for i in range(4)]
mu_s0 = np.mean(Data_df[Data_df.Image == images[0]].norm_Rating)
sigma_s = [np.std(Data_df[Data_df.Image == images[i+1]].norm_Rating) for i in range(4)]
sigma_s0 = np.std(Data_df[Data_df.Image == images[0]].norm_Rating)
fprs, tprs = [], []

for i in range(4):
    fpr, tpr = roc(Data_df[Data_df.Image == images[0]].norm_Rating, Data_df[Data_df.Image == images[i+1]].norm_Rating, (mu_s[i]+mu_s0)/2)
    fprs.append(fpr)
    tprs.append(tpr)


In [ ]:
print(mu_s0)
sigma_s0

In [ ]:
from sympy import symbols, solve, Eq

real_cs = []

for i in range(len(tprs)):
    x = symbols('x')
    if tprs[i][0] == 1.0:
        Eq1 = Eq(-1/sigma_s[i]*x+mu_s[i]/sigma_s[i], norm.ppf(0.9999999999, loc=mu_s[i], scale=sigma_s[i])) #eq 1.7 lecture notes 2
    else:
        Eq1 = Eq(-1/sigma_s[i]*x+mu_s[i]/sigma_s[i], norm.ppf(tprs[i], loc=mu_s[i], scale=sigma_s[i])[0])
    sol = solve(Eq1)
    print(sol)
    real_cs.append(sol[0])

real_cs

In [ ]:
synthetic_range = np.linspace(-0.2, 1.2, 5)
correct_responses = []

for i,img in enumerate(sorted(Data_df.Image.unique())):
    print(img)
    temp_data = Data_df[Data_df.Image == img]
    correct = temp_data.norm_Rating >= real_cs[0]
    print(np.sum(correct))
    correct_responses.append(np.sum(correct))

correct_responses = [float(i)/max(correct_responses) for i in correct_responses]
number_of_correct_responses = correct_responses
print(number_of_correct_responses)